In [3]:
# LangChain + RAG 예시 코드 (뉴스 QA, 검색 청크 출력)
import os
from typing import List
from dotenv import load_dotenv

# 문서 로더, 텍스트 분할기(청킹), 벡터스토어(FAISS)
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS

# OpenAI 임베딩 및 LLM
from langchain_openai import OpenAIEmbeddings, ChatOpenAI

# 대화형 메시지 포맷(시스템 지시 / 사용자 메시지)
from langchain.schema import HumanMessage, SystemMessage


In [4]:
# 0. API Key 설정 (.env에서 불러오기)
load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")

if not api_key:
    raise ValueError("OPENAI_API_KEY가 설정되지 않았습니다. .env 파일을 확인하세요.")

ValueError: OPENAI_API_KEY가 설정되지 않았습니다. .env 파일을 확인하세요.

In [ ]:
# 1. 텍스트 파일 로드
def load_news_data(txt_file_path):
    """
    텍스트 파일을 LangChain의 Document 형태로 로드합니다.
    - TextLoader는 파일을 읽어서 Document 리스트로 반환합니다.
    - 각 Document에는 'page_content'(텍스트)와 'metadata'(부가정보)가 포함됩니다.
    """
    if not os.path.exists(txt_file_path):
        raise FileNotFoundError(f"파일을 찾을 수 없습니다: {txt_file_path}")

    loader = TextLoader(txt_file_path, encoding="utf-8")
    documents = loader.load()
    print(f"로드 완료: {len(documents)}개 문서")

    return documents

In [ ]:
# 2. 문서 분할 (청킹)
def split_documents(documents, chunk_size = 200, chunk_overlap = 50):
    """
    긴 문서를 작은 청크로 분할합니다.
    - chunk_size: 한 청크의 최대 길이 (문자 수 기준).
    - chunk_overlap: 청크 간 겹치는 부분의 길이. 문맥 단절을 방지합니다.
    """
    splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap
    )
    splits = splitter.split_documents(documents)
    print(f"분할 완료: {len(splits)}개 청크 생성")

    # 디버깅용: 앞의 몇 개 청크 미리보기
    preview_n = min(3, len(splits))
    for i in range(preview_n):
        print(f"\n[청크 미리보기 {i+1}]")
        print(splits[i].page_content[:200].strip(), "...")
    return splits

In [ ]:
# 3. 벡터 DB 구축 (임베딩 → FAISS 인덱스)
def store_in_vector_db(splits):
    """
    텍스트 청크를 임베딩(벡터화)한 후, FAISS를 이용해 빠른 유사도 검색이 가능한 DB를 만듭니다.
    - OpenAIEmbeddings: OpenAI의 임베딩 모델을 사용합니다.
    - FAISS: 대규모 벡터 검색에 최적화된 라이브러리입니다.
    """
    embeddings = OpenAIEmbeddings(api_key=api_key, model="text-embedding-3-small")
    vector_store = FAISS.from_documents(splits, embeddings)
    print("벡터 DB 생성 완료")
    return vector_store

In [ ]:
# 4. 문서 검색
def retrieve_similar_docs(query_text, vector_store, k = 2):
    """
    사용자의 질문을 벡터화하여, 의미적으로 가장 유사한 청크 k개를 검색합니다.
    """
    docs = vector_store.similarity_search(query_text, k=k)

    # 검색된 청크 출력
    print("\n[검색된 청크]")
    for i, d in enumerate(docs, 1):
        print(f"\n--- 청크 {i} ---")
        print(d.page_content.strip())
    return docs

In [ ]:
# 5. 답변 생성 (검색된 청크 기반)
def generate_answer(query_text, docs):
    """
    검색된 청크를 LLM에 제공하여 답변을 생성합니다.
    - SystemMessage: 모델의 역할과 규칙 정의 (예: 뉴스 분석 전문가).
    - HumanMessage: 실제 질문과 검색된 청크를 함께 전달.
    - LLM은 "주어진 청크"만 근거로 답변합니다.
    """
    llm = ChatOpenAI(model="gpt-4o-mini", temperature=0, api_key=api_key)

    # 검색된 청크들을 하나의 텍스트로 합칩니다.
    docs_text = "\n\n".join([doc.page_content for doc in docs])

    system_message = SystemMessage(
        content="너는 뉴스 분석 전문가야. 반드시 주어진 문서 내용만 바탕으로 답변하고, 문서에 없는 내용은 '잘 모르겠습니다'라고 말해."
    )
    human_message = HumanMessage(
        content=f"질문: {query_text}\n\n[참고 문서]\n{docs_text}"
    )

    response = llm.invoke([system_message, human_message])
    return response.content

In [ ]:
# 6. 실행
txt_file_path = "./news.txt"  # 분석할 뉴스 기사 파일

queries = [
    "정부가 발표한 총 예산 규모는 얼마인가?",
    "농촌 지역 교통격차 해소를 위해 얼마가 투입되었나?",
    "친환경 버스 보급과 관련된 목표는 무엇인가?",
]

# Step 1. 뉴스 데이터 로드
documents = load_news_data(txt_file_path)

# Step 2. 문서 분할 (청킹)
splits = split_documents(documents, chunk_size=200, chunk_overlap=50)

# Step 3. 벡터 DB 구축
vector_store = store_in_vector_db(splits)

# Step 4. 질문별 검색 및 답변 생성
for q in queries:
    print("\n" + "-" * 80)
    print("질문:", q)

    similar_docs = retrieve_similar_docs(q, vector_store, k=2)
    answer = generate_answer(q, similar_docs)

    print("\n답변:", answer)